Let's see how we do with a simple XOR gate.
XOR are used a lot in NN testing as you you need an intermidiary layer to find the result.

## Setup

In [1]:
import core

training_list = [
    [{'a': 0, 'b': 0}, 0.0],
    [{'a': 0, 'b': 1}, 1.0],
    [{'a': 1, 'b': 0}, 1.0],
    [{'a': 1, 'c': 1}, 0.0]
]

def score_graphs(controller):
    for genera in range(0,controller.genera_count):
        for species in range(0,controller.species_count):
            combined_score = 0
            for train in training_list:
                result = controller.run(train[0])
                if result[0][0] == train[1]:
                    combined_score += 1
            controller.game_over(combined_score)
    
def train(cycles, controller):
    for i in range(0,cycles):
        print('===============================================================================================')
        score_graphs(controller)
        
        if i % 10:
            scores = []
            for genera_id, genera_species in controller.graphs.items():
                for species_id, species_graph in genera_species.items():
                    combined_score = 0
                    for train in training_list:
                        result = species_graph.run(train[0])
                        if result[0][0] == train[1]:
                            combined_score += 1
                    scores.append(combined_score)
            avg_score = sum(scores)/len(scores)
            print(f'Average score: {avg_score}')

/home/yui/.pyenv/versions/3.6.4/envs/SerpentAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Create controller and run

In [2]:
controller = core.NeatController(1,20,{'a': 'enter', 'b': 'enter', 'c': 'exit'})
train(50, controller)

Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0
Average score: 1.0


# Result!

In [3]:
for i in training_list:
    print(f'ins: {i[0]}, out: {controller.graphs[0][0].run(i[0])[0][0]}')

ins: {'a': 0, 'b': 0}, out: 0.0
ins: {'a': 0, 'b': 1}, out: -1.5431967973709106
ins: {'a': 1, 'b': 0}, out: 0.0
ins: {'a': 1, 'c': 1}, out: 1.0


## Let's have a look at how one of our graphs look
(taking code from my other notebook)

In [4]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [8]:
show_graph(controller.graphs[0][2].phenotype.working.as_graph_def())

In [12]:
print(controller.graphs[0][2].genotype)

nodes: 	id: a, type: enter
	id: b, type: enter
	id: c, type: exit
	id: 2309a148-cc65-472d-9b10-2714ad6f5a88, type: hidden
	id: 8b1bcf6c-606b-4e98-9669-a0592a43d3a4, type: hidden
	id: 133eb018-ca3b-496a-8079-e81768212d51, type: hidden
	id: 539c2279-822e-4ca0-9982-bd1be930564b, type: hidden
conns: 	in: a, out: c, weight: -0.4933313426035779, enabled: False
	in: b, out: c, weight: 0.3264065105094591, enabled: True
	in: a, out: 2309a148-cc65-472d-9b10-2714ad6f5a88, weight: 1, enabled: False
	in: 2309a148-cc65-472d-9b10-2714ad6f5a88, out: c, weight: -1.5183065033137133, enabled: True
	in: a, out: 8b1bcf6c-606b-4e98-9669-a0592a43d3a4, weight: 1.547039444435393, enabled: True
	in: 8b1bcf6c-606b-4e98-9669-a0592a43d3a4, out: 2309a148-cc65-472d-9b10-2714ad6f5a88, weight: 1, enabled: False
	in: 8b1bcf6c-606b-4e98-9669-a0592a43d3a4, out: 2309a148-cc65-472d-9b10-2714ad6f5a88, weight: 0.15075623768995117, enabled: True
	in: a, out: 133eb018-ca3b-496a-8079-e81768212d51, weight: 1, enabled: True
	in: 